# Демонстрация использования класса `OscillogramFinder`

Этот ноутбук предназначен для демонстрации возможностей класса `OscillogramFinder` из `filesystem.finder`. `OscillogramFinder` используется для обнаружения, копирования и фильтрации файлов осциллограмм различных форматов, включая обработку архивов и предотвращение дубликатов с помощью хеширования.

Будут рассмотрены следующие функции:
- Копирование новых осциллограмм (`copy_new_oscillograms`) с различными параметрами (выбор типов файлов, сохранение структуры директорий, использование хешей).
- Поиск осциллограмм по номерам терминалов на основе JSON-метаданных (`find_terminal_hashes_from_json`).
- Поиск осциллограмм, содержащих каналы тока нулевой последовательности (`find_oscillograms_with_neutral_current`).

In [ ]:
import os
import shutil
import json
import hashlib
import datetime
import zipfile
import py7zr 
try:
    import aspose.zip as az
    HAS_ASPOSE_ZIP = True
except ImportError:
    HAS_ASPOSE_ZIP = False
    print("Предупреждение: Библиотека 'aspose.zip' не найдена. Обработка RAR-архивов будет недоступна.")

import pandas as pd 
import sys

# Настройка путей для импорта модулей проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from filesystem.finder import OscillogramFinder, TYPE_OSC
from core.oscillogram import Oscillogram 

# --- Создание временных директорий ---
base_sample_dir_finder = "temp_sample_data_finder"
source_dir_main = os.path.join(base_sample_dir_finder, "source_main")
source_dir_archives_temp = os.path.join(base_sample_dir_finder, "archives_temp") # Временная для создания архивов
dest_dir_finder = os.path.join(base_sample_dir_finder, "destination_oscillograms")
config_dir_finder = os.path.join(base_sample_dir_finder, "config_finder") 

if os.path.exists(base_sample_dir_finder):
    shutil.rmtree(base_sample_dir_finder)
os.makedirs(source_dir_main)
os.makedirs(source_dir_archives_temp)
os.makedirs(dest_dir_finder)
os.makedirs(config_dir_finder)

# --- Вспомогательные функции для создания файлов ---
def create_dummy_file(filepath, content="dummy_content"):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w", encoding="utf-8") as f:
        # Добавляем путь к содержимому для уникальности хеша, если хешируем сам файл
        # Для DAT файлов, содержимое важно для хеша.
        f.write(content + "_" + filepath)

def create_sample_cfg_for_neutral_check(path, name, has_neutral_current_signal):
    cfg_content = "Station,Dev,1999\n2,2A,0D\n"
    cfg_content += "1,Ua,A,,V,1,0,0,0,100,1,1,P\n"
    if has_neutral_current_signal:
        cfg_content += "2,I | Bus Bar 1 | phase: N,N,,A,1,0,0,0,100,1,1,P\n"
    else:
        cfg_content += "2,Ib,B,,A,1,0,0,0,100,1,1,P\n"
    cfg_content += "50.0\n1\n1000,100\n01/01/2023,00:00:00.000000\n01/01/2023,00:00:01.000000\nASCII\n1.0\n"
    dat_content = "1,0,10,20\n2,1000,12,22\n"
    with open(os.path.join(path, f"{name}.cfg"), "w", encoding="utf-8") as f: f.write(cfg_content)
    with open(os.path.join(path, f"{name}.dat"), "w", encoding="utf-8") as f: f.write(dat_content)

# --- Наполнение source_dir_main файлами разных типов ---
create_dummy_file(os.path.join(source_dir_main, "comtrade_pair1.cfg"), "cfg_content_1")
create_dummy_file(os.path.join(source_dir_main, "comtrade_pair1.dat"), "dat_content_1")
create_dummy_file(os.path.join(source_dir_main, "subdir1", "comtrade_pair2.cfg"), "cfg_content_2")
create_dummy_file(os.path.join(source_dir_main, "subdir1", "comtrade_pair2.dat"), "dat_content_2")
create_dummy_file(os.path.join(source_dir_main, "comtrade_single.cff"), "cff_content")
create_dummy_file(os.path.join(source_dir_main, "bresler_file" + OscillogramFinder(is_print_message=False).BRESLER_EXTENSION), "bresler_content")
create_dummy_file(os.path.join(source_dir_main, "subdir2", "ekra_file" + OscillogramFinder(is_print_message=False).EKRA_EXTENSION), "ekra_content")
create_dummy_file(os.path.join(source_dir_main, "report.txt"), "report_content")
create_dummy_file(os.path.join(source_dir_main, "subdir1", "notes.doc"), "doc_content")

create_sample_cfg_for_neutral_check(source_dir_main, "neutral_present", True)
create_sample_cfg_for_neutral_check(os.path.join(source_dir_main, "subdir2"), "neutral_absent", False)

# --- Создание архивов во временной папке archives_temp ---
zip_path = os.path.join(source_dir_archives_temp, "archive1.zip")
with zipfile.ZipFile(zip_path, 'w') as zf:
    zf.writestr("archived_cfg_in_zip.cfg", "cfg_content_zip1")
    zf.writestr("archived_dat_in_zip.dat", "dat_content_zip1") # Этот DAT будет хешироваться
    zf.writestr("folder_in_zip/another.brs", "brs_content_zip")

seven_zip_path = os.path.join(source_dir_archives_temp, "archive2.7z")
with py7zr.SevenZipFile(seven_zip_path, 'w') as szf:
    szf.writestr("archived_cff_in_7z.cff", "cff_content_7z")

rar_path = os.path.join(source_dir_archives_temp, "archive3.rar")
if HAS_ASPOSE_ZIP:
    try:
        # Создаем RAR с одним файлом cfg (aspose.zip может быть ограничен в создании сложных архивов без лицензии)
        temp_rar_source_dir = os.path.join(source_dir_archives_temp, "temp_rar_source")
        os.makedirs(temp_rar_source_dir, exist_ok=True)
        with open(os.path.join(temp_rar_source_dir, "file_for_rar.cfg"), "w") as f_rar_cfg: f_rar_cfg.write("cfg_in_rar")
        with open(os.path.join(temp_rar_source_dir, "file_for_rar.dat"), "w") as f_rar_dat: f_rar_dat.write("dat_in_rar_for_hash")
        shutil.make_archive(os.path.splitext(rar_path)[0], 'zip', temp_rar_source_dir) # Create a zip first
        # Rename zip to rar for aspose to open; actual RAR creation is complex
        # This is a workaround. For real RAR test, place a RAR file manually.
        # Aspose.zip's `Archive` class can open various formats including RAR for extraction.
        # For this demo, we'll just ensure a .rar file exists. OscillogramFinder will try to process it.
        # If aspose.zip is fully functional, it should extract. Otherwise, it'll be skipped by finder.
        # Actually creating a valid RAR is hard without dedicated tools or full Aspose license.
        # Let's just create a dummy file if aspose is not for creation, or an empty rar.
        with az.Archive() as archive: archive.save(rar_path)
        print(f"Создан (пустой) RAR-архив: {rar_path})")
    except Exception as e_rar:
        print(f"Не удалось создать RAR-архив с aspose.zip: {e_rar}. Создан пустой файл-заглушка.")
        create_dummy_file(rar_path, "dummy rar content")
else:
    create_dummy_file(rar_path, "dummy rar content")

# Перемещаем архивы в основную директорию поиска
for item in os.listdir(source_dir_archives_temp):
    shutil.move(os.path.join(source_dir_archives_temp, item), source_dir_main)
shutil.rmtree(source_dir_archives_temp)

# Создадим начальный _hash_table.json
initial_hashes_path = os.path.join(dest_dir_finder, "_hash_table.json")
dat1_hash_for_initial = hashlib.md5(b"dat_content_zip1").hexdigest()
initial_hashes_content = { dat1_hash_for_initial: "archived_dat_in_zip.dat" } # Path is relative to dest_dir / type_folder / preserve_structure_path
with open(initial_hashes_path, "w", encoding="utf-8") as f: json.dump(initial_hashes_content, f)

print(f"Создана директория с исходными файлами и архивами: {source_dir_main}")
print(f"Директория назначения: {dest_dir_finder}")

## 1. Инициализация `OscillogramFinder`

In [ ]:
finder = OscillogramFinder(is_print_message=True)

## 2. Копирование новых осциллограмм (`copy_new_oscillograms`)
Этот метод является основным. Он рекурсивно обходит исходную директорию, находит файлы осциллограмм (включая те, что в архивах), и копирует их в директорию назначения, избегая дубликатов на основе хеш-сумм (если включено).

In [ ]:
print("\n--- Демонстрация 1: Копирование только COMTRADE (.cfg/.dat, .cff) с сохранением структуры ---")
dest_dir_run1 = os.path.join(base_sample_dir_finder, "destination_run1")
if os.path.exists(dest_dir_run1): shutil.rmtree(dest_dir_run1)
os.makedirs(dest_dir_run1)

# Скопируем initial_hashes в эту директорию, чтобы finder его нашел
if os.path.exists(initial_hashes_path):
    shutil.copy2(initial_hashes_path, os.path.join(dest_dir_run1, "_hash_table.json"))

file_types_to_copy_cfg_cff = { type_enum: False for type_enum in TYPE_OSC } 
file_types_to_copy_cfg_cff[TYPE_OSC.COMTRADE_CFG_DAT] = True
file_types_to_copy_cfg_cff[TYPE_OSC.COMTRADE_CFF] = True
file_types_to_copy_cfg_cff[TYPE_OSC.ARCH_ZIP] = True 
file_types_to_copy_cfg_cff[TYPE_OSC.ARCH_7Z] = True
file_types_to_copy_cfg_cff[TYPE_OSC.ARCH_RAR] = True

copied_count1 = finder.copy_new_oscillograms(
    source_dir=source_dir_main,
    dest_dir=dest_dir_run1,
    copied_hashes_input=None, # Заставим его загрузить _hash_table.json из dest_dir_run1
    preserve_dir_structure=True,
    use_hashes=True,
    file_type_flags=file_types_to_copy_cfg_cff,
    max_archive_depth=2
)
print(f"Скопировано файлов (попытка 1): {copied_count1}")
print(f"Содержимое директории назначения ({dest_dir_run1}):")
for root, dirs, files in os.walk(dest_dir_run1):
    level = root.replace(dest_dir_run1, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files: print(f"{sub_indent}{f_name}")

if os.path.exists(os.path.join(dest_dir_run1, "_hash_table.json")):
    with open(os.path.join(dest_dir_run1, "_hash_table.json"), "r") as f_ht:
        print("\nСодержимое _hash_table.json:")
        try: display(pd.DataFrame(list(json.load(f_ht).items()), columns=['Hash', 'RelativePath']))
        except NameError: print(json.dumps(json.load(f_ht), indent=2, ensure_ascii=False))
        
newly_copied_files_run1 = [f for f in os.listdir(dest_dir_run1) if f.startswith("newly_copied_hashes_")]
if newly_copied_files_run1:
    with open(os.path.join(dest_dir_run1, newly_copied_files_run1[0]), "r") as f_nht:
        print(f"\nСодержимое {newly_copied_files_run1[0]}:")
        try: display(pd.DataFrame(list(json.load(f_nht).items()), columns=['Hash', 'RelativePath']))
        except NameError: print(json.dumps(json.load(f_nht), indent=2, ensure_ascii=False))

In [ ]:
print("\n--- Демонстрация 2: Копирование всех типов, без сохранения структуры, без начальных хешей ---")
dest_dir_run2 = os.path.join(base_sample_dir_finder, "destination_run2")
if os.path.exists(dest_dir_run2): shutil.rmtree(dest_dir_run2)
os.makedirs(dest_dir_run2)

copied_count2 = finder.copy_new_oscillograms(
    source_dir=source_dir_main,
    dest_dir=dest_dir_run2,
    copied_hashes_input=None, 
    preserve_dir_structure=False, 
    use_hashes=True, 
    file_type_flags=None, # Все типы
    max_archive_depth=2
)
print(f"Скопировано файлов (попытка 2): {copied_count2}")
print(f"Содержимое директории назначения ({dest_dir_run2}):")
for root, dirs, files in os.walk(dest_dir_run2):
    level = root.replace(dest_dir_run2, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files: print(f"{sub_indent}{f_name}")

## 3. Поиск осциллограмм по номерам терминалов (`find_terminal_hashes_from_json`)
Этот статический метод анализирует JSON-файл, который содержит информацию о путях к файлам осциллограмм, и находит хеши, связанные с указанными номерами терминалов (по конвенциям именования файлов или путей).

In [ ]:
input_meta_json_path = os.path.join(config_dir_finder, "input_metadata.json")
path_to_comtrade_pair1_dat = os.path.join(source_dir_main, "comtrade_pair1.dat")
path_to_neutral_present_dat = os.path.join(source_dir_main, "neutral_present.dat")

dat1_hash = ""
if os.path.exists(path_to_comtrade_pair1_dat):
    with open(path_to_comtrade_pair1_dat, "rb") as f: dat1_hash = hashlib.md5(f.read()).hexdigest()
dat_neutral_hash = ""
if os.path.exists(path_to_neutral_present_dat):
    with open(path_to_neutral_present_dat, "rb") as f: dat_neutral_hash = hashlib.md5(f.read()).hexdigest()

input_meta_content = {
    dat1_hash: ["comtrade_pair1.cfg", "some/path/ОТГРУЖЕННЫЕ ТЕРМИНАЛЫ И ШКАФЫ/00123/comtrade_pair1.cfg"],
    "hash_example_t00456": ["t00456_other.brs", "another/path/t00456_other.brs"],
    dat_neutral_hash: ["neutral_present.cfg", "ОТГРУЖЕННЫЕ ТЕРМИНАЛЫ И ШКАФЫ/00123/extra/neutral_present.cfg"],
    "dummy_hash_no_match": ["random_file.err", "some/other/path/file.err"]
}
with open(input_meta_json_path, "w", encoding="utf-8") as f: json.dump(input_meta_content, f, indent=4)

output_terminal_hashes_path = os.path.join(config_dir_finder, "terminal_hashes_output.json")
terminals_to_find = [123, 456, 789] 

print(f"\nПоиск хешей для терминалов: {terminals_to_find}")
OscillogramFinder.find_terminal_hashes_from_json(
    input_json_path=input_meta_json_path,
    terminal_numbers_to_find=terminals_to_find,
    output_json_path=output_terminal_hashes_path,
    is_print_message=True
)

if os.path.exists(output_terminal_hashes_path):
    print(f"Содержимое {output_terminal_hashes_path}:")
    with open(output_terminal_hashes_path, "r", encoding="utf-8") as f:
        try: display(pd.DataFrame(list(json.load(f).items()), columns=['Terminal', 'Hashes']))
        except NameError: print(json.dumps(json.load(f), indent=2, ensure_ascii=False))

## 4. Поиск осциллограмм с каналами тока нулевой последовательности (`find_oscillograms_with_neutral_current`)
Этот метод сканирует `.cfg` файлы в указанной директории, загружает их с помощью `Oscillogram` и анализирует имена аналоговых каналов на предмет наличия признаков тока нейтрали (например, "I" и "N" в имени).

In [ ]:
neutral_current_report_path = os.path.join(config_dir_finder, "neutral_current_report.txt")

print(f"\nПоиск осциллограмм с током нейтрали в {source_dir_main}")
finder.find_oscillograms_with_neutral_current(
    cfg_directory=source_dir_main, 
    output_txt_path=neutral_current_report_path,
    is_print_message=True
)

if os.path.exists(neutral_current_report_path):
    print(f"Содержимое отчета о файлах с током нейтрали ({neutral_current_report_path}):")
    with open(neutral_current_report_path, "r", encoding="utf-8") as f:
        print(f.read())

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(base_sample_dir_finder):
        shutil.rmtree(base_sample_dir_finder)
        print(f"\nВременная директория {base_sample_dir_finder} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {base_sample_dir_finder}: {e}")